In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import os
import io
import geopandas as gpd
import seaborn as sns
import statsmodels.api as sm
from functools import reduce
import scipy.stats as stats
from collections import OrderedDict
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import random

# suppress warning

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
# display all columns
# avoid scientific digit
pd.options.display.float_format = '{:.8f}'.format

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
new_df = pd.read_csv('/content/drive/My Drive/X999/new_df.csv')

In [ ]:
new_df['building_age'] = 2024 - new_df['yearbuilt']

In [ ]:
new_df.drop(columns=['median_income'], inplace=True)

In [ ]:
new_df.rename(columns={'2023_median_income': 'median_income'}, inplace=True)

In [ ]:
new_df.drop(columns=['2005_median_income'], inplace=True)

In [ ]:
identifier_columns = ['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number',
 'executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year','month_year']
metrics_columns = ['eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']
building_columns = ['numfloors', 'unitsres', 'bldgarea', 'bldgclass','building_type', 'architectural_style', "building_age",
                    'building_category', 'is_condo','is_llc']
complaints_columns = ['air_quality', 'animal_issues', 'appliances',  'building_exterior', 'doors_windows', 'electrical_issues', 'elevator_issues',
'floors_stairs','general_complaints', 'graffiti_posting', 'heat_hot_water','homeless_issues', 'noise_complaints','other_issues', 'pest_issues',
'plumbing_issues', 'police_matters', 'public_nuisance','safety_concerns', 'sanitation_issues', 'walls_ceilings', 'total_complaints']
other_features_columns = ['avg_idr','avg_idr_change','income_change','income_change_ratio',
'median_income', 'rpl_theme1', 'rpl_theme2', 'rpl_theme3', 'rpl_theme4', 'rpl_themes', 'ep_pov150', 'ep_unemp', 'ep_nohsdp',
 'ep_uninsur', 'ep_age65', 'ep_age17', 'ep_disabl', 'ep_limeng', 'ep_noveh', 'ep_crowd', 'ep_hburd', 'ep_afam', 'ep_hisp', 'ep_asian',
 'ep_nhpi', 'ep_twomore', 'ep_otherrace', 'ep_minrty', 'ep_white']
# new_df['avg_change_abs']

In [ ]:
# idr_evi.columns.to_list()
all_columns = identifier_columns + metrics_columns + building_columns + complaints_columns + other_features_columns
len(all_columns)

84

In [ ]:
feature_columns = building_columns + complaints_columns + other_features_columns
len(feature_columns)

61

In [ ]:
new_df = new_df[all_columns]
new_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,D9,0,"1931–1950, Manhattan Modern",84.00000000,elevator,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,14.00000000,1861.00000000,1658649.00000000,D3,1,"1931–1950, Manhattan Modern",74.00000000,elevator,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000
2,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000,26.00000000,371.00000000,453600.00000000,D9,1,"1951–1980, the International Style, Alternative Modernism",58.00000000,elevator,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000
3,Lower East Side,0301867/23_6498,1003820001,0301867/23,6498,1 HAVEN PL

In [ ]:
new_df[feature_columns].head()

,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,6.00000000,813.00000000,513000.00000000,D9,0,"1931–1950, Manhattan Modern",84.00000000,elevator,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000
1,14.00000000,1861.00000000,1658649.00000000,D3,1,"1931–1950, Manhattan Modern",74.00000000,elevator,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000
2,26.00000000,371.00000000,453600.00000000,D9,1,"1951–1980, the International Style, Alternative Modernism",58.00000000,elevator,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000
3,26.00000000,371.00000000,453600.00000000,D9,1,"1951–1980, the International Style, Alternative Modernism",58.00000000,elevator,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000
4,5.00000000,20.00000000,9775.00000000,C1,0,"1900–1920, Beaux-Arts",124.00000000,walk-up,0,0,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,40.00000000,0.00000000,11.00000000,0.00000000,2.00000000,4.00000000,0.00000000,0.00000000,0.00000000,1.00000000,0.00000000,58.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00

In [ ]:
new_df.bldgclass.unique()

array(['D9', 'D3', 'C1', 'C4', 'C7', 'D1', 'D6', 'C9', 'D7', 'D4', 'S4',
       'C6', 'S5', 'S2', 'S3', 'C5', 'D5', 'R1', 'S9', 'D8', 'R4', 'C2',
       'RR', 'C3', 'C0', 'R2', 'B9', 'A4', 'D2', 'B3', 'B1', 'B2', 'A1',
       'A9', 'S1', 'A5', 'R3', 'A7', 'S0', 'D0', 'R9', 'A2', 'C8', 'A6',
       'A3', 'A8', 'A0', 'R5'], dtype=object)

In [ ]:
new_df.building_category.unique()

array(['elevator', 'walk-up', 'condo-co-op',
       'primarily_res_with_mixed_use', 'two-family', 'single-family'],
      dtype=object)

In [ ]:
unique_strings1 = new_df['bldgclass'].unique()
unique_strings2 = new_df['building_category'].unique()
unique_strings3 = new_df['architectural_style'].unique()
# unique_strings4 = new_df['decade'].unique()

string_to_label1 = {string: random.randint(1, 100) for string in unique_strings1}
string_to_label2 = {string: random.randint(1, 100) for string in unique_strings2}
string_to_label3 = {string: random.randint(1, 100) for string in unique_strings3}
# string_to_label4 = {string: random.randint(1, 100) for string in unique_strings4}

new_df['bldgclass_label'] = new_df['bldgclass'].map(string_to_label1)
new_df['building_category_label'] = new_df['building_category'].map(string_to_label2)
new_df['architectural_style_label'] = new_df['architectural_style'].map(string_to_label3)
# new_df['decade_label'] = new_df['decade'].map(string_to_label)

In [ ]:
# new_df.age.unique()

In [ ]:
# string_to_label4 = {'1940-1949': 1940, '1950-1959':1950, '1960-1969':1960, '1900-1909':1900, '1910-1919':1910, '1980-1989':1980, '1970-1979':1970, '1930-1939':1930, '1920-1929':1920, '1990-1999':1990, '2000-2009':2000, 'Pre-1900':1890, '2010-2020':2010}
# new_df['decade_label'] = new_df['decade'].map(string_to_label4)

In [ ]:
# new_df.building_age.unique()

In [ ]:
new_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,bldgclass_label,building_category_label,architectural_style_label
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,D9,0,"1931–1950, Manhattan Modern",84.00000000,elevator,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000,22,57,92
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,14.00000000,1861.00000000,1658649.00000000,D3,1,"1931–1950, Manhattan Modern",74.00000000,elevator,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,83,57,92
2,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000,26.00000000,371.00000000,453600.00000000,D9,1,"1951–1980, the International Style, Alternative Modernism",58.00000000,elevator,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000

In [ ]:
new_df.drop(columns=['bldgclass', 'building_category', 'architectural_style'], inplace=True)

In [ ]:
new_df.rename(columns={'bldgclass_label': 'bldgclass', 'building_category_label': 'building_category', \
                       'architectural_style_label': 'architectural_style'}, inplace=True)

In [ ]:
new_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,building_type,building_age,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,bldgclass,building_category,architectural_style
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,0,84.00000000,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000,22,57,92
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,14.00000000,1861.00000000,1658649.00000000,1,74.00000000,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,83,57,92
2,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000,26.00000000,371.00000000,453600.00000000,1,58.00000000,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,22,57,15
3,Lower East Side,0301867/23_6498,1003820001,0301867/23,6498,1 HAVEN PLAZA,27C,2023-06-20,MANHATTAN,10009,40.72701600,-73.97644000,3.00000000,2.00000000,28.00000000,1083141,Lower East Side,2023,2023-0

In [ ]:
new_df.columns

Index(['Location', 'primary_key', 'bbl', 'court_index_number', 'docket_number',
       'eviction_address', 'eviction_apartment_number', 'executed_date',
       'borough', 'zipcode', 'latitude', 'longitude', 'community_board',
       'council_district', 'census_tract', 'bin', 'nta', 'year', 'month_year',
       'eviction_count_nta', 'eviction_count_building', 'eviction_count_unit',
       'eviction_count_zipcode', 'numfloors', 'unitsres', 'bldgarea',
       'building_type', 'building_age', 'is_condo', 'is_llc', 'air_quality',
       'animal_issues', 'appliances', 'building_exterior', 'doors_windows',
       'electrical_issues', 'elevator_issues', 'floors_stairs',
       'general_complaints', 'graffiti_posting', 'heat_hot_water',
       'homeless_issues', 'noise_complaints', 'other_issues', 'pest_issues',
       'plumbing_issues', 'police_matters', 'public_nuisance',
       'safety_concerns', 'sanitation_issues', 'walls_ceilings',
       'total_complaints', 'avg_idr', 'avg_idr_change', '

## **Groupby zipcode on complaint features**

In [ ]:
complaints_columns

['air_quality',
 'animal_issues',
 'appliances',
 'building_exterior',
 'doors_windows',
 'electrical_issues',
 'elevator_issues',
 'floors_stairs',
 'general_complaints',
 'graffiti_posting',
 'heat_hot_water',
 'homeless_issues',
 'noise_complaints',
 'other_issues',
 'pest_issues',
 'plumbing_issues',
 'police_matters',
 'public_nuisance',
 'safety_concerns',
 'sanitation_issues',
 'walls_ceilings',
 'total_complaints']

In [ ]:
zipcode_df = new_df.groupby('zipcode')[complaints_columns].sum().reset_index()

In [ ]:
zipcode_df.head()

,zipcode,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints
0,10001,7.00000000,204.00000000,62.00000000,16.00000000,226.00000000,158.00000000,347.00000000,44.00000000,130.00000000,5.00000000,928.00000000,327.00000000,13041.00000000,11.00000000,193.00000000,638.00000000,64.00000000,17.00000000,93.00000000,338.00000000,234.00000000,17083.00000000
1,10002,227.00000000,947.00000000,222.00000000,14.00000000,1085.00000000,274.00000000,1798.00000000,347.00000000,824.00000000,42.00000000,6238.00000000,296.00000000,79758.00000000,98.00000000,1511.00000000,2839.00000000,906.00000000,303.00000000,209.00000000,1881.00000000,732.00000000,100551.00000000
2,10003,19.00000000,34.00000000,63.00000000,6.00000000,292.00000000,67.00000000,89.00000000,71.00000000,231.00000000,12.00000000,1509.00000000,240.00000000,3339.00000000,5.00000000,179.00000000,758.00000000,26.00000000,11.00000000,60.00000000,595.00000000,251.00000000,7857.00000000
3,10005,10.00000000,116.00000000,261.00000000,0.00000000,182.00000000,1333.00000000,291.00000000,18.00000000,956.00000000,0.00000000,1480.00000000,52.00000000,6294.00000000,0.00000000,95.00000000,1152.00000000,88.00000000,6.00000000,63.00000000,206.00000000,42.00000000,12645.00000000
4,10006,2.00000000,19.00000000,4.00000000,18.00000000,44.00000000,184.00000000,404.00000000,42.00000000,74.00000000,0.00000000,954.00000000,27.00000000,1651.00000000,4.00000000,10.00000000,448.00000000,38.00000000,22.00000000,33.00000000,228.00000000,56.00000000,4262.00000000


In [ ]:
zipcode_df = new_df.merge(zipcode_df, on='zipcode', how='left', suffixes=('', '_zipcode'))

In [ ]:
zipcode_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,building_type,building_age,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,bldgclass,building_category,architectural_style,air_quality_zipcode,animal_issues_zipcode,appliances_zipcode,building_exterior_zipcode,doors_windows_zipcode,electrical_issues_zipcode,elevator_issues_zipcode,floors_stairs_zipcode,general_complaints_zipcode,graffiti_posting_zipcode,heat_hot_water_zipcode,homeless_issues_zipcode,noise_complaints_zipcode,other_issues_zipcode,pest_issues_zipcode,plumbing_issues_zipcode,police_matters_zipcode,public_nuisance_zipcode,safety_concerns_zipcode,sanitation_issues_zipcode,walls_ceilings_zipcode,total_complaints_zipcode
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,0,84.00000000,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000,22,57,92,227.00000000,947.00000000,222.00000000,14.00000000,1085.00000000,274.00000000,1798.00000000,347.00000000,824.00000000,42.00000000,6238.00000000,296.00000000,79758.00000000,98.00000000,1511.00000000,2839.00000000,906.00000000,303.00000000,209.00000000,1881.00000000,732.00000000,100551.00000000
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,14.00000000,1861.00000000,1658649.00000000,1,74.00000000,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,83,57,92,265.00000000,2464.00000000,957.00000000,62.00000000,2750.00000000,643.00000000,2209.00000000,1639.00000000,1308.00000000,33.00000000,35535.00000000,1220.00000000,117786.00000000,100.00000000,2620.00000000,7773.00000000,498.00000000,779.00000000,324.00000000,12839.00000000,2546.00000000,194350.00000

In [ ]:
zipcode_df = zipcode_df.drop(columns= feature_columns)
# remeber to drop

In [ ]:
zipcode_df.rename(columns={'air_quality_zipcode': 'air_quality', 'animal_issues_zipcode': 'animal_issues', 'appliances_zipcode': 'appliances', \
                           'building_exterior_zipcode': 'building_exterior', 'doors_windows_zipcode': 'doors_windows', 'electrical_issues_zipcode': 'electrical_issues', \
                           'elevator_issues_zipcode': 'elevator_issues', 'floors_stairs_zipcode': 'floors_stairs', 'general_complaints_zipcode': 'general_complaints', \
                           'graffiti_posting_zipcode': 'graffiti_posting', 'heat_hot_water_zipcode': 'heat_hot_water', 'homeless_issues_zipcode': 'homeless_issues', \
                           'noise_complaints_zipcode': 'noise_complaints', 'other_issues_zipcode': 'other_issues', 'pest_issues_zipcode': 'pest_issues', \
                           'plumbing_issues_zipcode': 'plumbing_issues', 'police_matters_zipcode': 'police_matters', 'public_nuisance_zipcode': 'public_nuisance', \
                           'safety_concerns_zipcode': 'safety_concerns', 'sanitation_issues_zipcode': 'sanitation_issues', 'walls_ceilings_zipcode': 'walls_ceilings', \
                           'total_complaints_zipcode': 'total_complaints'}, inplace=True)

In [ ]:
zipcode_df[complaints_columns] = zipcode_df[complaints_columns].div(5, axis=0)

In [ ]:
zipcode_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,45.40000000,189.40000000,44.40000000,2.80000000,217.00000000,54.80000000,359.60000000,69.40000000,164.80000000,8.40000000,1247.60000000,59.20000000,15951.60000000,19.60000000,302.20000000,567.80000000,181.20000000,60.60000000,41.80000000,376.20000000,146.40000000,20110.20000000
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,53.00000000,492.80000000,191.40000000,12.40000000,550.00000000,128.60000000,441.80000000,327.80000000,261.60000000,6.60000000,7107.00000000,244.00000000,23557.20000000,20.00000000,524.00000000,1554.60000000,99.60000000,155.80000000,64.80000000,2567.80000000,509.20000000,38870.00000000
2,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000,53.00000000,492.80000000,191.40000000,12.40000000,550.00000000,128.60000000,441.80000000,327.80000000,261.60000000,6.60000000,7107.00000000,244.00000000,23557.20000000,20.00000000,524.00000000,1554.60000000,99.60000000,155.80000000,64.80000000,2567.80000000,509.20000000,38870.00000000
3,Lower East Side,0301867/23_6498,1003820001,0301867/23,6498,1 HAVEN PLAZA,27C,2023-06-20,MANHATTAN,10009,40.72701600,-73.97644000,3.00000000,2.00000000,28.00000000,1083141,Lower East Side,2023,2023-06,56.72749658,0.80000000,0.00215633,67.00000000,53.00000000,492.80000000,191.40000000,12.40000000,550.00000000,128.60000000,441.80000000,327.80000000,261.60000000,6.60000000,7107.00000000,244.00000000,23557.20000000,20.00000000,524.00000000,1554.60000000,99.60000000,155.80000000,64.80000000,2567.80000000,509.20000000,38870.00000000
4,Lower East Side,0308915/24_212303,1004350013,0308915/24,212303,98 ST MARKS PLACE,5B,2024-10-22,MANHATTAN,10009,40.72739900,-73.98485400,3.00000000,2.00000000,32.00000000,1005824,East Village,2024,2024-10,56.72749658,0.20000000,0.01000000,67.00000000,53.00000000,492.80000000,191.40000000,12.40000000,550.00000000,128.60000000,441.80000000,327.80000000,261.60000000,6.60000000,7107.00000000,244.00000000,23557.20000000,20.00000000,524.00000000,1554.60000000,99.60000000,155.80000000,64.80000000,2567.80000000,509.20000000,38870.00000000


## **Groupby building features at zipcode level**

In [ ]:
building_columns

['numfloors',
 'unitsres',
 'bldgarea',
 'bldgclass',
 'building_type',
 'architectural_style',
 'building_age',
 'building_category',
 'is_condo',
 'is_llc']

In [ ]:
new_df.head(1)

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,building_type,building_age,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,bldgclass,building_category,architectural_style
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,0,84.00000000,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000,22,57,92


In [ ]:
# lambda x: x.mode().iloc[0]

In [ ]:
zipcode_df_building = new_df.groupby('zipcode').agg({'building_age': "mean",
                                                     'numfloors': 'mean',
                                                     "unitsres": "mean",
                                                     "bldgarea":"mean",
                                                     'bldgclass': lambda x: x.mode().iloc[0],
                                                     'building_type': lambda x: x.mode().iloc[0],
                                                     'architectural_style': lambda x: x.mode().iloc[0],
                                                     'building_category': lambda x: x.mode().iloc[0],
                                                     'is_condo': lambda x: x.mode().iloc[0],
                                                     'is_llc': lambda x: x.mode().iloc[0],
                                                     }).reset_index()

In [ ]:
zipcode_df_building.head()

,zipcode,building_age,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_category,is_condo,is_llc
0,10001,67.07594937,18.46835443,259.36708861,288214.50000000,13,1,97,57,0,1
1,10002,86.35409836,11.56065574,446.47213115,497571.38360656,93,0,97,57,0,0
2,10003,107.01449275,7.65217391,64.79710145,55548.33333333,93,0,97,83,0,0
3,10005,73.47916667,25.02083333,445.18750000,423269.39583333,60,1,15,57,0,1
4,10006,79.46666667,23.20000000,344.86666667,298641.66666667,60,1,15,57,0,1


In [ ]:
zipcode_df = new_df.merge(zipcode_df_building, on='zipcode', how='left', suffixes=('', '_building'))

In [ ]:
zipcode_df.head()

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,numfloors,unitsres,bldgarea,building_type,building_age,is_condo,is_llc,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,bldgclass,building_category,architectural_style,building_age_building,numfloors_building,unitsres_building,bldgarea_building,bldgclass_building,building_type_building,architectural_style_building,building_category_building,is_condo_building,is_llc_building
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,813.00000000,513000.00000000,0,84.00000000,0,0,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000,22,57,92,86.35409836,11.56065574,446.47213115,497571.38360656,93,0,97,57,0,0
1,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,10009,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000,14.00000000,1861.00000000,1658649.00000000,1,74.00000000,0,0,0.00000000,16.00000000,0.00000000,0.00000000,6.00000000,0.00000000,14.00000000,0.00000000,0.00000000,0.00000000,0.00000000,10.00000000,1021.00000000,1.00000000,24.00000000,13.00000000,4.00000000,11.00000000,1.00000000,58.00000000,0.00000000,1179.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8.30000000,78.40000000,3.90000000,28.50000000,6.90000000,24.60000000,13.40000000,0.00000000,4.30000000,0.60000000,49.90000000,50.10000000,83,57,92,88.62388060,9.93731343,1680.33134328,1668334.22985075,83,1,92,57,0,0
2,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,10009,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000,26.00000000,371.00000000,453600.00000000,1,58.00000000,0,0,2.00000000,1.00000000,4.00000000,0.00000000,15.00000000,15.00000000,26.00000000,6.00000000,11.00000000,0.00000000,92.00000000,0.00000000,90.00000000,0.00000000,4.00000000,31.00000000,0.00000000,1.00000000,2.00000000,32.00000000,12.00000000,344.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.78560000,0.48210000,0.86760000,0.95160000,0.87210000,29.60000000,4.80000000,13.30000000,2.80000000,15.70000000,10.50000000,14.00000000,8

In [ ]:
zipcode_df = zipcode_df.drop(columns= building_columns)

In [ ]:
zipcode_df.rename(columns={'building_age_building': 'building_age', 'numfloors_building': 'numfloors', 'unitsres_building': 'unitsres', 'bldgarea_building': 'bldgarea', "bldgclass_building": 'bldgclass',
                           'building_type_building': 'building_type', 'architectural_style_building': 'architectural_style', 'building_category_building': 'building_category', \
                           'is_condo_building': 'is_condo', 'is_llc_building': 'is_llc'}, inplace=True)

In [ ]:
zipcode_df.head(1)

,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,building_age,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_category,is_condo,is_llc
0,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,10002,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000,6.00000000,2.00000000,0.00000000,0.00000000,3.00000000,0.00000000,4.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,942.00000000,4.00000000,10.00000000,10.00000000,9.00000000,4.00000000,0.00000000,1.00000000,0.00000000,998.00000000,10.59498500,7.27757147,5956.73487000,12.27553287,54482.00000000,0.96390000,0.87810000,0.93690000,0.92540000,0.96560000,36.80000000,7.60000000,30.00000000,5.40000000,23.10000000,13.10000000,13.00000000,24.70000000,82.80000000,6.80000000,40.80000000,8.00000000,26.00000000,37.20000000,0.00000000,2.40000000,0.80000000,74.40000000,25.60000000,86.35409836,11.56065574,446.47213115,497571.38360656,93,0,97,57,0,0


In [ ]:
zipcode_df.columns.to_list(), zipcode_df.shape

(['Location',
  'primary_key',
  'bbl',
  'court_index_number',
  'docket_number',
  'eviction_address',
  'eviction_apartment_number',
  'executed_date',
  'borough',
  'zipcode',
  'latitude',
  'longitude',
  'community_board',
  'council_district',
  'census_tract',
  'bin',
  'nta',
  'year',
  'month_year',
  'eviction_count_nta',
  'eviction_count_building',
  'eviction_count_unit',
  'eviction_count_zipcode',
  'air_quality',
  'animal_issues',
  'appliances',
  'building_exterior',
  'doors_windows',
  'electrical_issues',
  'elevator_issues',
  'floors_stairs',
  'general_complaints',
  'graffiti_posting',
  'heat_hot_water',
  'homeless_issues',
  'noise_complaints',
  'other_issues',
  'pest_issues',
  'plumbing_issues',
  'police_matters',
  'public_nuisance',
  'safety_concerns',
  'sanitation_issues',
  'walls_ceilings',
  'total_complaints',
  'avg_idr',
  'avg_idr_change',
  'income_change',
  'income_change_ratio',
  'median_income',
  'rpl_theme1',
  'rpl_theme2',
  

In [ ]:
final_model = sm.OLS.from_formula(
    "np.log(eviction_count_zipcode + 1) ~ avg_idr + avg_idr_change + ep_afam + income_change + median_income + ep_hisp + ep_hburd + is_llc + ep_pov150 + rpl_themes + unitsres + building_age + total_complaints",
    data=zipcode_df
).fit()
print(final_model.summary())

                                    OLS Regression Results                                    
Dep. Variable:     np.log(eviction_count_zipcode + 1)   R-squared:                       0.766
Model:                                            OLS   Adj. R-squared:                  0.765
Method:                                 Least Squares   F-statistic:                 1.667e+04
Date:                                Tue, 10 Jun 2025   Prob (F-statistic):               0.00
Time:                                        19:36:02   Log-Likelihood:                -39244.
No. Observations:                               66395   AIC:                         7.852e+04
Df Residuals:                                   66381   BIC:                         7.864e+04
Df Model:                                          13                                         
Covariance Type:                            nonrobust                                         
                       coef    std err          t 

In [ ]:
zipcode_df.to_csv('/content/drive/My Drive/X999/zipcode_df.csv', index=False)

In [ ]:
len(pd.DataFrame(zipcode_df).columns.to_list())

84

In [ ]:
columns = complaints_columns + building_columns + other_features_columns
len(columns)

61

## **Zipcode_df normalized**

In [ ]:
scaler = StandardScaler()

In [ ]:
zipcode_df_normalized = zipcode_df.copy()
zipcode_df_normalized = scaler.fit_transform(zipcode_df_normalized[complaints_columns + building_columns + other_features_columns])
zipcode_df_normalized = pd.DataFrame(zipcode_df_normalized, columns=complaints_columns + building_columns + other_features_columns)

In [ ]:
zipcode_df_normalized.head(1)

,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,2.63255622,0.10182496,-0.24441651,-0.23014407,-0.35120112,-0.41575982,-0.12829407,-0.45963993,-0.36198386,-0.20618317,-0.28308591,0.54446416,1.63664822,12.00138064,0.10621251,-0.26712625,0.30834208,1.10000168,-0.43745134,-0.39014153,-0.41691376,0.88264038,1.44206581,0.47110497,0.45858918,1.71124008,-0.48218687,1.25805475,0.22376957,-0.82605988,-0.07404125,-0.68750473,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,0.33656949,0.39990329,-0.13447419,-0.08025525,0.31459399,0.37685617,-0.56794790,1.07678865,-0.68257460,2.20956395,-1.80374947,0.02659191,1.40547024,1.27492190,-0.79928722,-0.18133595,-0.95824164,-0.55596050,2.41678554,-0.25158118,-0.28234398,-0.15525459,-0.32955230,0.32955230


In [ ]:
df = zipcode_df['zipcode']
df.head()

,zipcode
0,10002
1,10009
2,10009
3,10009
4,10009


In [ ]:
identifier_columns = ['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number',
 'executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year','month_year']
metrics_columns = ['eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']

In [ ]:
zipcode_df_normalized['zipcode'] = zipcode_df['zipcode']
zipcode_df_normalized[['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number','executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year',
                      'month_year','eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']] = zipcode_df[['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number','executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year',
                      'month_year','eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']]
zipcode_df_normalized.head()

,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,zipcode,Location,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,eviction_count_nta,eviction_count_building,eviction_count_unit,eviction_count_zipcode
0,2.63255622,0.10182496,-0.24441651,-0.23014407,-0.35120112,-0.41575982,-0.12829407,-0.45963993,-0.36198386,-0.20618317,-0.28308591,0.54446416,1.63664822,12.00138064,0.10621251,-0.26712625,0.30834208,1.10000168,-0.43745134,-0.39014153,-0.41691376,0.88264038,1.44206581,0.47110497,0.45858918,1.71124008,-0.48218687,1.25805475,0.22376957,-0.82605988,-0.07404125,-0.68750473,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,0.33656949,0.39990329,-0.13447419,-0.08025525,0.31459399,0.37685617,-0.56794790,1.07678865,-0.68257460,2.20956395,-1.80374947,0.02659191,1.40547024,1.27492190,-0.79928722,-0.18133595,-0.95824164,-0.55596050,2.41678554,-0.25158118,-0.28234398,-0.15525459,-0.32955230,0.32955230,10002,Lower East Side,014679/18_115084,1002600001,014679/18,115084,630 WATER STREET,05E,2019-03-15,MANHATTAN,40.71138100,-73.98286600,3.00000000,2.00000000,202.00000000,1077523,Lower East Side,2019,2019-03,56.72749658,0.60000000,0.00073801,61.00000000
1,-0.21785384,2.46930182,-0.24441651,-0.23014407,-0.23378550,-0.41575982,0.10719624,-0.45963993,-0.36198386,-0.20618317,-0.28308591,2.03610482,1.79042197,2.86113171,0.36455922,-0.23116852,0.09503932,3.24885496,-0.23214575,0.39008113,-0.41691376,1.11511494,0.98541046,2.68364222,2.18646591,1.20268377,2.07388476,1.12782242,0.42979777,-0.82605988,-0.07404125,-0.68750473,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998,10009,Lower East Side,015506/17_112423,1003560001,015506/17,112423,711 FDR DR,11E,2018-08-22,MANHATTAN,40.72071700,-73.97489500,3.00000000,2.00000000,20.00000000,1078059,Lower East Side,2018,2018-08,56.72749658,0.80000000,0.00042988,67.00000000
2,0.73228284,-0.06728053,-0.07592603,-0.23014407,0.11846136,0.60546195,0.38978462,-0.06594408,0.17064092,-0.20618317,0.08827574,-0.09481040,-0.02177249,-0.18561793,-0.00450751,-0.01542219,-0.07560289,0.17906456,-0.02684016,0.03419009,-0.14815433,0.04264942,0.98541046,2.68364222,2.18646591,1.20268377,2.07388476,1.12782242,0.42979777,-0.82605988,-0.07404125,-0.68750473,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998,10009,Lower East Side,0300290/22_5780,1003820001,0300290/22,5780,3 HAVEN PLAZA,15D,2023-05-26,MANHATTAN,40.72682400,-73.97598600,3.00000000,2.00000000,28.00000000,1083144,Lower East Side,2023,2023-05,56.72749658,0.80000000,0.00215633,67.00000000
3,0.73228284,-0.06728053,-0.07592603,-0.23014407,0.11846136,0

In [ ]:
# last_column = zipcode_df_normalized.pop('zipcode')
# zipcode_df_normalized.insert(0, 'zipcode', last_column)
# zipcode_df_normalized.head()

In [ ]:
columns_to_move = ['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number','executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year',
                      'month_year','eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']

In [ ]:
for col in columns_to_move:
    last_column = zipcode_df_normalized.pop(col)
    zipcode_df_normalized.insert(0, col, last_column)
zipcode_df_normalized.head()

,eviction_count_zipcode,eviction_count_unit,eviction_count_building,eviction_count_nta,month_year,year,nta,bin,census_tract,council_district,community_board,longitude,latitude,zipcode,borough,executed_date,eviction_apartment_number,eviction_address,docket_number,court_index_number,bbl,primary_key,Location,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,61.00000000,0.00073801,0.60000000,56.72749658,2019-03,2019,Lower East Side,1077523,202.00000000,2.00000000,3.00000000,-73.98286600,40.71138100,10002,MANHATTAN,2019-03-15,05E,630 WATER STREET,115084,014679/18,1002600001,014679/18_115084,Lower East Side,2.63255622,0.10182496,-0.24441651,-0.23014407,-0.35120112,-0.41575982,-0.12829407,-0.45963993,-0.36198386,-0.20618317,-0.28308591,0.54446416,1.63664822,12.00138064,0.10621251,-0.26712625,0.30834208,1.10000168,-0.43745134,-0.39014153,-0.41691376,0.88264038,1.44206581,0.47110497,0.45858918,1.71124008,-0.48218687,1.25805475,0.22376957,-0.82605988,-0.07404125,-0.68750473,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,0.33656949,0.39990329,-0.13447419,-0.08025525,0.31459399,0.37685617,-0.56794790,1.07678865,-0.68257460,2.20956395,-1.80374947,0.02659191,1.40547024,1.27492190,-0.79928722,-0.18133595,-0.95824164,-0.55596050,2.41678554,-0.25158118,-0.28234398,-0.15525459,-0.32955230,0.32955230
1,67.00000000,0.00042988,0.80000000,56.72749658,2018-08,2018,Lower East Side,1078059,20.00000000,2.00000000,3.00000000,-73.97489500,40.72071700,10009,MANHATTAN,2018-08-22,11E,711 FDR DR,112423,015506/17,1003560001,015506/17_112423,Lower East Side,-0.21785384,2.46930182,-0.24441651,-0.23014407,-0.23378550,-0.41575982,0.10719624,-0.45963993,-0.36198386,-0.20618317,-0.28308591,2.03610482,1.79042197,2.86113171,0.36455922,-0.23116852,0.09503932,3.24885496,-0.23214575,0.39008113,-0.41691376,1.11511494,0.98541046,2.68364222,2.18646591,1.20268377,2.07388476,1.12782242,0.42979777,-0.82605988,-0.07404125,-0.68750473,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998
2,67.00000000,0.00215633,0.80000000,56.72749658,2023-05,2023,Lower East Side,1083144,28.00000000,2.00000000,3.00000000,-73.97598600,40.72682400,10009,MANHATTAN,2023-05-26,15D,3 HAVEN PLAZA,5780,0300290/22,1003820001,0300290/22_5780,Lower East Side,0.73228284,-0.06728053,-0.07592603,-0.23014407,0.11846136,0.60546195,0.38978462,-0.06594408,0.17064092,-0.20618317,0.08827574,-0.09481040,-0.02177249,-0.18561793,-0.00450751,-0.01542219,-0.07560289,0.17906456,-0.02684016,0.03419009,-0.14815433,0.04264942,0.98541046,2.68364222,2.18646591,1.20268377,2.07388476,1.12782242,0.42979777,-0.82605988,-0.07404125,-0.68750473,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998
3,67.00000000,0.00215633,0.80000000,56.72749658,2023-06,2023,

In [ ]:
last_column = zipcode_df_normalized.pop('zipcode')
zipcode_df_normalized.insert(0, 'zipcode', last_column)
zipcode_df_normalized.head()

,zipcode,eviction_count_zipcode,eviction_count_unit,eviction_count_building,eviction_count_nta,month_year,year,nta,bin,census_tract,council_district,community_board,longitude,latitude,borough,executed_date,eviction_apartment_number,eviction_address,docket_number,court_index_number,bbl,primary_key,Location,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,10002,61.00000000,0.00073801,0.60000000,56.72749658,2019-03,2019,Lower East Side,1077523,202.00000000,2.00000000,3.00000000,-73.98286600,40.71138100,MANHATTAN,2019-03-15,05E,630 WATER STREET,115084,014679/18,1002600001,014679/18_115084,Lower East Side,2.63255622,0.10182496,-0.24441651,-0.23014407,-0.35120112,-0.41575982,-0.12829407,-0.45963993,-0.36198386,-0.20618317,-0.28308591,0.54446416,1.63664822,12.00138064,0.10621251,-0.26712625,0.30834208,1.10000168,-0.43745134,-0.39014153,-0.41691376,0.88264038,1.44206581,0.47110497,0.45858918,1.71124008,-0.48218687,1.25805475,0.22376957,-0.82605988,-0.07404125,-0.68750473,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,0.33656949,0.39990329,-0.13447419,-0.08025525,0.31459399,0.37685617,-0.56794790,1.07678865,-0.68257460,2.20956395,-1.80374947,0.02659191,1.40547024,1.27492190,-0.79928722,-0.18133595,-0.95824164,-0.55596050,2.41678554,-0.25158118,-0.28234398,-0.15525459,-0.32955230,0.32955230
1,10009,67.00000000,0.00042988,0.80000000,56.72749658,2018-08,2018,Lower East Side,1078059,20.00000000,2.00000000,3.00000000,-73.97489500,40.72071700,MANHATTAN,2018-08-22,11E,711 FDR DR,112423,015506/17,1003560001,015506/17_112423,Lower East Side,-0.21785384,2.46930182,-0.24441651,-0.23014407,-0.23378550,-0.41575982,0.10719624,-0.45963993,-0.36198386,-0.20618317,-0.28308591,2.03610482,1.79042197,2.86113171,0.36455922,-0.23116852,0.09503932,3.24885496,-0.23214575,0.39008113,-0.41691376,1.11511494,0.98541046,2.68364222,2.18646591,1.20268377,2.07388476,1.12782242,0.42979777,-0.82605988,-0.07404125,-0.68750473,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998
2,10009,67.00000000,0.00215633,0.80000000,56.72749658,2023-05,2023,Lower East Side,1083144,28.00000000,2.00000000,3.00000000,-73.97598600,40.72682400,MANHATTAN,2023-05-26,15D,3 HAVEN PLAZA,5780,0300290/22,1003820001,0300290/22_5780,Lower East Side,0.73228284,-0.06728053,-0.07592603,-0.23014407,0.11846136,0.60546195,0.38978462,-0.06594408,0.17064092,-0.20618317,0.08827574,-0.09481040,-0.02177249,-0.18561793,-0.00450751,-0.01542219,-0.07560289,0.17906456,-0.02684016,0.03419009,-0.14815433,0.04264942,0.98541046,2.68364222,2.18646591,1.20268377,2.07388476,1.12782242,0.42979777,-0.82605988,-0.07404125,-0.68750473,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998
3,10009,67.00000000,0.00215633,0.80000000,56.72749658,2023-06

In [ ]:
# last_columns = zipcode_df_normalized.pop('Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number','executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year',
#                       'month_year','eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode')
# zipcode_df_normalized.insert(1, last_columns)
# zipcode_df_normalized.head()

In [ ]:
zipcode_df_normalized.to_csv('/content/drive/My Drive/X999/zipcode_df_normalized.csv', index=False)

In [ ]:
# type(zipcode_df_normalized['eviction_count_zipcode'] = np

In [ ]:
final_model = sm.OLS.from_formula(
    "np.log(eviction_count_zipcode + 1) ~ avg_idr + avg_idr_change + ep_afam + income_change + median_income + ep_hisp + ep_hburd + is_llc + ep_pov150 + rpl_themes + unitsres + building_age + total_complaints",
    data=zipcode_df_normalized
).fit()
print(final_model.summary())

                                    OLS Regression Results                                    
Dep. Variable:     np.log(eviction_count_zipcode + 1)   R-squared:                       0.766
Model:                                            OLS   Adj. R-squared:                  0.765
Method:                                 Least Squares   F-statistic:                 1.667e+04
Date:                                Tue, 10 Jun 2025   Prob (F-statistic):               0.00
Time:                                        19:36:22   Log-Likelihood:                -39244.
No. Observations:                               66395   AIC:                         7.852e+04
Df Residuals:                                   66381   BIC:                         7.864e+04
Df Model:                                          13                                         
Covariance Type:                            nonrobust                                         
                       coef    std err          t 